# 20 Questions

*Ryan Becwar and Matthew Dragan*

## Table of Contents
* [20 Questions](#20-Questions)
	* [Inroduction](#Introduction)
	* [Methods](#Methods)
      * [twentyQ Class](#twentyQ-Class)
      * [UI Class](#UI-Class)
    * [Results](#Results)
      * [Playing an object that is known by the game](#Playing-an-object-that-is-known-by-the-game)
      * [Playing an object that is not known by the game](#Playing-an-object-that-is-not-known-by-the-game)
    * [Conclusion](#Conclusion)
    * [What each group member did](#What-each-group-member-did)
    * [References](#References)

## Introduction
An intelligent agent should be able to make descisions given some information.  In addition, such an agent should be able to learn about a new subject given some information on that subject.  In this project we will try to produce a simplified version of the described agent by creating a program that can learn about new items by asking questions about them. 

The agent will play a version of the 20 questions game. 20 questions is a game in which a computer will try to guess what the player is thinking based on some simple 'yes' or 'no' questions.  In this version the goal objects will consist of fruits and vegetables, and the questions will be specialized for differentiating between fruits and vegetables. 
Additionally, we will try to produce a version of the game that has the following characteristics:
  * The game will determine which question to ask next based on which question most evenly divides the remaining possibilities and which question is most decisive.  This means that if the game has narrowed the possibilities down to 10 fruits and vegetables then the game will try to pick a question that has 5 'yes' answers and 5 'no' answers and has the most decisive weights (meaning that the question has consistently had the same answer so the game can be more certain).  
  * If the player plays a fruit or vegetable that the game has not heard of before then the game will add that fruit or vegetable to the database and use the players answers as the 'truth' about that fruit or vegetable.  For example, if the player is playing 'raspberry' and the game has never heard of 'raspberry' before then 'raspberry' will be added to the database and the answers that the player gave will be considered to be the 'truth' about raspberries.  If the player chooses a fruit or vegetable that hasn't been played before then player is asked to answer all questions about that fruit or vegetable so the answers will be added to the database.  
  * If there are two fruits/vegetables that are hard to tell apart based on the questions (radishes and beets for example) the game will pick the response based on which has the highest weight, and the weights are updated each time the object is played.  
  * The game will be able to get to the correct answer even if the player answers a question wrong.  
  * The game will be able to add a different value for each question.  This means that a question that is frequently answered differently will not be valued as high.  This will allow the game handle uncertainties between players.      
  
If implemented correctly, and with enough practice, the game should be able to frequently guess the correct fruit or vegetable based on the player's input.  


## Methods

We chose to use a decision tree inspired methodology to implement the Twenty Questions Game. Based on the answers the user has given us so far, we ask questions which divide the answer statespace as much as possible. We determine this score through two factors: a division score which selects for a similar number of yes/no answers, and a decisive score which selects for questions which have a low number of indeterminate or 'maybe' answers. This leads to the same branching behavior you would see in a pre-built tree structure, however only one branch is actually explored per game.

Early in our design process, we found that building an actual tree structure would become cumbersome, inflexible, and require significantly more memory usage than our methods would.  This could, however, potentially reduce the running time for as the question selection step for our method has an O(nm) running time, with n being the number of items, and m being the number of questions. We did not find this to be an issue however, as with our dataset computation is instantaneous, and computation time is buffered by the fact that it is a user facing application.  We expect that it would remain instantaneous even with the full dataset of a standard 20 questions game.

We implemented a learning process to update the weights, or expected answers to each question for each item when the user states that their item is different from our guess.  Our implementation of this allows us to easily insert new items into the statespace, and for the user to provide uncertain answers such as 'maybe' and 'sometimes'.  We found this to be particularly vital, as a user's implementation of questions such as 'Can it be found in a pie' may be quite different than ours. This is primarily done through the `updateWeights(answer)` method described below.

In order to create the game we implemented a twentyQ class in python which will implement all of the functions required to run the game and a user interface class UI which will allow a player to play the game.



### twentyQ Class
This class contains the required functions to implement the 20 Questions game.
The twentyQ class contains the following functions:
  * constructor
    * (Brief) - This function initializes the twenty questions class.  
    * (Details) - This function initializes all of the class variables of the twenty questions class, calls the $readData()$ class function to read data from files, and calls the $processData()$ class function to set up the data for use.  
    The class variables are as follows:
      * `answers` (dictionary) - This dictionary holds the answers to each question for each fruit and vegetable. The keys are the possible objects the player could be thinking of (ex. apple, carrot, etc.) and the values are array lists of answers for specific questions for that specific key.  
      * `likelihood` (dictionary) - This dictionary holds the likelihood that an object (ex. apple, carrot, etc.) is the object that the player is thinking of.  The keys are the objects the player could be thinking of and the values are a float giving the likelihood for each object.  
      * `prevAnswers` (dictionary) - This dictionary holds a 0 to 1 value that represents the mean of past yes/no answers to each question.   The keys are the objects and the values are a list which holds the mean of past yes/no answers to each question.
      * `timesPlayed` (dictionary) - For each object, this dictionary holds the number of times that question has been played.  The keys into the dictionary are the objects and the values are array lists that hold the number of times that question has been played for that specific object.  
      * `questions` (array list) - An array list that holds the list of questions as strings.  
      * `questionsUsed` (array list) - An array that holds the index of the questions that have been asked.  These indeces correspond with the `questions` array.  
      * `remainingFood` (array list) - An array list that holds the foods that are still considered to be possible answers.
      * `answersGiven` (array list) - An array list that holds the answers given by the player.  These answers correspond to the `questionsUsed` array list.  
    * (Inputs) - None
    * (Outputs) - None
    * (Returns) - None
    * (Notes) - Relies on $processData()$ function and $readData()$ function.


  * $readData()$
    * (Brief) Reads the data from CSV files required to play the game. 
    * (Details) This function opens the three CSV files required to play the game into three different data frames using the `pandas` library.  Then the function separates the questions from one of the data frames and stores them in the class variable `questions`.  Then The function separates the data from each dataframe and stores them in variables
    * (Inputs) - None
    * (Outputs) - None
    * (Returns) - returns a tuple containing three matrices.  The first matrix contains the objects and the answers to the corresponding questions, the second matrix contains the objects and their previous answers, and the third matrix contains the objects and the number of times a question has been asked for that object.  For each matrix the first column contains the objects and the remaining columns contain the values.
    * (Notes) - Relies on the `pandas` library and the `numpy` library.  
    
    
  * $processData()$
    * (Brief) - This function sets up the data received from the $readData()$ function so it can be used by the game.   
    * (Details) - This function sets up the `answers` dictionary, the `prevAnswers` dictionary, and the `timesPlayed` dictionary using the matrices that were returned by the $readData()$ function.
    * (Inputs) - 
      * `data` (numpy matrix) the first value in the tuple returned by $readData()$
      * `prevAnswers` (numpy matrix) the second value in the tuple returned by $readData()$
      * `times` (numpy matix) the second value in the tuple returned by $readData()$
    * (Outputs) - the `answers`, `prevAnswers`, and `timesPlayed` dictionaries will all be filled in. 
    * (Returns) - None
    * (Notes) - None
    
    
  * $resetGame()$
    * (Brief) - resets the game. 
    * (Details) - This function will be used to restart the game if the user wants to restart by calling the constuctor again.  
    * (Inputs) - None
    * (Outputs) - None
    * (Returns) - None
    * (Notes) - Relies on the class constructor
    
  * $getNextQuestion()$
    * (Brief) Gets the best unasked question
    * (Details) Determines the question which most effectively divides the items in the `remainingFood` list.  To do this, it considers all unasked questions, calculates a score for how dividing that question is (similar number of yes/no answers) and for how decisive the question is (sum of difference from neutral .5 score, inverted), and aggregates these two values. If there are two questions with an equl score, it will select one at random.
    * (Inputs) None
    * (Outputs) None
    * (Returns) The text of the best question
    * (Notes) Relies on the `TwentyQ` class

  * $convertAnswer()$
    * (Brief) - This function converts a input string from the player to a representative value
    * (Details) - This function takes an input from the user.  If the player inputs a 'yes' or 'y' the function returns a 1, if the player inputs a 'no' or an 'n' the function returns a 0, if the player inputs a 'sometimes', 'maybe', 'unknown', or 's' the function will return 0.5, and if the player inputs any other value the function will return a -1 to indictate that the value has not been understood. 
    * (Inputs) - currentA (string) the answer input by the player. 
    * (Outputs) - None
    * (Returns) - Returns the value that represents the input of the user
    * (Notes) - None
    
  * $answerQuestion(currentQ, currentA)$
    * (Brief) Updates the class to reflect an answered question
    * (Details) Calls the `updateLikelihood` method, then updates `remainingFood` to all answers which have a likelihood within THRESHOLD of the highest likelihood answer
    * (Inputs)
      * `CurrentQ` (Integer) Index of current question
      * `CurrentA` (Float) The 0-1 value of answer corresponding to no-yes
    * (Outputs) None
    * (Returns) None
    * (Notes) Relies on the `TwentyQ` class

  * $updateLikelihood(currentQ, currentA)$
    * (Brief) Updates the likelihood dictionary
    * (Details) Adds 1 - the difference between the given answer and the expected answer for each item
    * (Inputs)
      * `CurrentQ` (Integer) Index of current question
      * `CurrentA` (Float) The 0-1 value of answer corresponding to no-yes
    * (Outputs) None
    * (Returns) None
    * (Notes) Relies on the `TwentyQ` class

  * $updateWeights(answer)$
    * (Brief) Learning step to update the weights table
    * (Details) If the item already exists in the weights table(`prevAnswers`), updates the value to the average of previous answers and the new answer. Otherwise, create a new entry in the weights table with initial values corresponding to the given answer. Calls `writeToCSV()` to save the changes.
    * (Inputs)
      * `answer` (String) Item the answer corresponds to.
    * (Outputs) None
    * (Returns) None
    * (Notes) Relies on the `TwentyQ` class
    
  * $askAnotherQuestion()$
    * (Brief) - Finds a question in the list of unasked questions and returns it.
    * (Details) - This function will find all the questions that have not been answered and return the first of these questions.  This function does not use any criteria to determine the next question.
    * (Inputs) - None
    * (Outputs) - None
    * (Returns) - Returns one of the remaining set of questions.  
    * (Notes) - None
    
    
  * $writeToCSV()$
    * (Brief) - This function writes `answers`, `prevAnswers`, and `timesPlayed` back to their respective CSV files.
    * (Details) - This function will write data back to a CSV including any updates that have been made to the values while the game has been running.  This allows the games performance to improve as time goes on.  
    * (Inputs) - None
    * (Outputs) - Will overwrite the three specified csv files.  
    * (Returns) - None
    * (Notes) - Relies on the `copy` library and `csv` library.  

### UI Class
This class implements the user interface of the twenty questions game.  
The UI class contains the following Functions:
  * constructor
    * (Brief) - Initializes the UI class
    * (Details) - Initializes the UI class by creating an instance of the twenty questions game.  
    * (Inputs) - None
    * (Outputs) - None
    * (Returns) - None
    * (Notes) - Relies on the `twentyQ` class.
    
    
  * $getRemainingUnanswered()$
    * (Brief) - This function asks the player the remaining questions if the object the player had been thinking of had not been added to the data base.  
    * (Details) - This function will ask the player the remaining questions and store the responses so the game can learn about a new object that it has not heard of before.  
    * (Inputs) - None
    * (Outputs) - Will prompt the player to answer the given question.  
    * (Returns) - None
    * (Notes) - Relies on the `twentyQ` class.  
    
    
    
  * $playGame()$
    * (Brief) - Plays the 20Q game with the user.  
    * (Details) - This function will play 20Q with the user repeatedly until the user until the user decides to quit playing.  The player will be prompted with a question about what he or she is thinking and the function will wait until the player gives an answer.  The expected answers are 'yes', 'no', 'maybe', 'sometimes', and 'unknown'.  The function will continue asking questions until it has run out of questions or some answer is believed based on a given threshold.  Then the user will be prompted with a guess.  If the guess is correct the player will be asked if he or she would like to play again.  If the guess is incorrect then the player will be asked what he or she was thinking of.  If the game has not heard of the users object the game will continue asking questions to learn more about the object.  Each time the game is played the weights of each question answer combination are updated so that the game gets better the longer it has been played.    
    * (Inputs) - None
    * (Outputs) - The function will output to the monitor to prompt the user for an answer as well as welcome them to the game. 
    * (Returns) - None
    * (Notes) - Relies on the `twentyQ` class and the $getRemainingUnanswered()$ function.

## Results

### Playing an object that is known by the game
Below is an example of the game selecting the correct answer playing the known object 'banana'.  Each time a question is answered we can see the how the likelihood is updated in the list of tuples.  The first element of the tuple is the object and the second element is the likelihood that the object is what the player is thinking.  If a certain object's likelihood is too low (the difference between the largest likelihood and the likelihood for a given object is larger than a specified threshold) then it is dropped from the list.  For example, we can see that several of the possibilities are dropped from the list after the game asks "Is it a fruit?".  Additionally, the game can reintroduce possibilities if the likelihood of an object grows to be within the threshold.  For example, we see 'apple' has re-emerged as a possibility afte the game asks "Is it commonly found on a sandwich?".  Additionally, the game is capable of getting the correct answer even if one (or possibly a few) questions have been answered incorrectly.  For example, the question "Is it crunchy when raw?" was answered with a 'yes', but this question should have been answered with a 'no'.  Even though one question was answered incorrectly the game was able to come up with 'banana' as the most likely answer (although, it may have taken a few more questions to determine the answer). 

In [5]:
import numpy as np
import pandas as pd
import csv
import copy as cp
import random
from twentyQ import *
from UI import *

In [6]:
me = UI()
me.playGame()

Welcome to 20 Questions!
Does it grow on a tree?
y
[('apple', 1.0), ('orange', 1.0), ('peach', 1.0), ('strawberry', 0.0), ('raspberry', 0.0), ('carrot', 0.0), ('potato', 0.0), ('lettuce', 0.0), ('onion', 0.0), ('broccoli', 0.0), ('banana', 1.0), ('jalepeno', 0.0), ('avocado', 1.0), ('celery', 0.0), ('grapefruit', 1.0), ('beet', 0.0), ('persimmon', 1.0), ('pineapple', 0.0), ('coconut', 1.0), ('spinach', 0.0), ('kale', 0.0), ('garlic', 0.0), ('radish', 0.0), ('cucumber', 0.0), ('pumpkin', 0.0), ('zucchini', 0.0), ('blueberry', 0.0), ('bell pepper', 0.0), ('tomato', 0.0), ('cantaloupe', 0.0), ('peanut', 0.0), ('almond', 1.0), ('pecan', 1.0), ('cauliflower', 0.0)]
Would you cook it in a pan?
n
[('apple', 2.0), ('orange', 2.0), ('peach', 2.0), ('strawberry', 1.0), ('raspberry', 1.0), ('carrot', 0.0), ('potato', 0.0), ('lettuce', 1.0), ('onion', 0.0), ('broccoli', 0.0), ('banana', 2.0), ('jalepeno', 0.0), ('avocado', 2.0), ('celery', 0.0), ('grapefruit', 2.0), ('beet', 0.0), ('persimmon', 2.

### Playing an object that is not known by the game
Below the game is played with an object that was not already known (dragon fruit).  Dragon fruit is not in the list of possibilities, so the game narrows the results down as much as it can and guesses persimmon as the most likely candidtate.  When the player says persimmon was not the object that he or she was thinking of the game prompts the player for the correct object.  The game then takes the player's answer 'dragon fruit' and adds it to the data set.  The player then plays again with the 'dragon fruit' as the object and we see that the game has learned to guess 'dragon fruit'.  The game is able to learn new fruits and vegetables as the game is played.  

In [7]:
me = UI()
me.playGame()

Welcome to 20 Questions!
Does it grow on a tree?
y
[('apple', 1.0), ('orange', 1.0), ('peach', 1.0), ('strawberry', 0.0), ('raspberry', 0.0), ('carrot', 0.0), ('potato', 0.0), ('lettuce', 0.0), ('onion', 0.0), ('broccoli', 0.0), ('banana', 1.0), ('jalepeno', 0.0), ('avocado', 1.0), ('celery', 0.0), ('grapefruit', 1.0), ('beet', 0.0), ('persimmon', 1.0), ('pineapple', 0.0), ('coconut', 1.0), ('spinach', 0.0), ('kale', 0.0), ('garlic', 0.0), ('radish', 0.0), ('cucumber', 0.0), ('pumpkin', 0.0), ('zucchini', 0.0), ('blueberry', 0.0), ('bell pepper', 0.0), ('tomato', 0.0), ('cantaloupe', 0.0), ('peanut', 0.0), ('almond', 1.0), ('pecan', 1.0), ('cauliflower', 0.0)]
Is it crunchy when raw?
n
[('apple', 1.0), ('orange', 2.0), ('peach', 2.0), ('strawberry', 1.0), ('raspberry', 1.0), ('carrot', 0.0), ('potato', 0.666666666666667), ('lettuce', 1.0), ('onion', 0.0), ('broccoli', 0.0), ('banana', 1.6666666666666667), ('jalepeno', 0.0), ('avocado', 2.0), ('celery', 0.0), ('grapefruit', 2.0), ('beet

[('apple', 3.0), ('orange', 4.0), ('peach', 4.0), ('strawberry', 4.0), ('raspberry', 4.0), ('potato', 1.666666666666667), ('lettuce', 2.0), ('banana', 3.666666666666667), ('avocado', 2.0), ('grapefruit', 4.0), ('persimmon', 4.0), ('pineapple', 3.0), ('coconut', 3.0), ('garlic', 2.0), ('pumpkin', 2.0), ('blueberry', 4.0), ('tomato', 2.5), ('cantaloupe', 4.0), ('almond', 2.0), ('pecan', 3.0), ('dragon fruit', 4.0)]
Does it grow on a tree?
y
[('apple', 4.0), ('orange', 5.0), ('peach', 5.0), ('strawberry', 4.0), ('raspberry', 4.0), ('banana', 4.666666666666667), ('avocado', 3.0), ('grapefruit', 5.0), ('persimmon', 5.0), ('pineapple', 3.0), ('coconut', 4.0), ('blueberry', 4.0), ('tomato', 2.5), ('cantaloupe', 4.0), ('almond', 3.0), ('pecan', 4.0), ('dragon fruit', 5.0)]
Can it be found in a pie?
n
[('apple', 4.0), ('orange', 6.0), ('peach', 5.0), ('strawberry', 4.0), ('raspberry', 4.0), ('banana', 4.666666666666667), ('avocado', 4.0), ('grapefruit', 6.0), ('persimmon', 6.0), ('pineapple', 4

## Conclusion
The game satisfied the desired objectives.  The game is able to learn about new fruits/vegetables from a player.  The game is able to get the right answer even if the player answers a question or two wrong.  The game can learn to weight certain question and answer combinations to determine how accurate an answer is.  This means the game will more likely ask questions where it is more certain of the answer.  Overall the game is capable of guessing the correct fruit or vegetable accurately and it can learn new fruits and vegetables as the game is played more. 

Some future ideas might be to expand this beyond fruits and vegetables.  We considered how we may be able to build on this idea to implement a more full game of 20 questions by adding a more decision tree structure.  We could begin the game by giving the player a set of categories to choose from and depending on which category we choose we could decide which dataset would likely have the result.  We may also consider trying some different methods such as neural networks to implement the game.  According to the original patent of the 20 questions game the creater used a nerual network to implement the game.  


## What each group member did
Ryan:
  * Developed the method of weighting answers. 
  * Developed methods that used weighting to determine which question to ask next.
  * Developed method of updating weights each time an object is played
  * Worked on UI interface
  * Documented several functions
  * Documented design of the project
  * Added items to dataset
 
Matt:
  * Developed structure of twentyQ class
  * Implemented file IO functions
  * Worked on UI interface
  * Wrote Results sections
  * Documented several functions
  * Added items to dataset
  
We both did about the same ammount of work on the project as a whole.  

## References
Bachman, Philip, et al. “Towards Information-Seeking Agents.” ICLR 2017, 8 Dec. 2016, arxiv.org/pdf/1612.02605.pdf.
Burgener, Robin. Artificial Neural Network Guessing Method and Game . 12 Oct. 2006. 